In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import os.path as osp
import numpy as np

In [ ]:
from segments import SegmentsClient

In [ ]:
api_key = "b40bd5dba051369bbc18735fb23925c456eaad7b"

client = SegmentsClient(api_key)

In [ ]:
def check_stamp(stamp):
    stamp_split = stamp.split(".")
    if len(stamp_split[-1]) < 9:
        stamp_split[-1] = "0" * (9 - len(stamp_split[-1])) + stamp_split[-1]
    return ".".join(stamp_split)

In [ ]:
# get timestamps

scene_id = 2

pcd_path = f"/media/kluo/My Passport/v2x_dataset/PointCloud/mini_{scene_id}"
odo_path = f"/media/kluo/My Passport/v2x_dataset/Odometry/mini_{scene_id}"

# store all filepaths according to their header
all_filepaths = {}
for f in os.listdir(pcd_path):
    if f.split(".")[-1] != "pcd":
        continue
    sensor, ts, stamp = f.split("_")
    stamp = check_stamp(stamp[:-4])
    stamp = float(stamp)
    if sensor not in all_filepaths:
        all_filepaths[sensor] = {ts: (f, stamp)}
    else:
        all_filepaths[sensor][ts] = (f, stamp)
all_sensors_list = list(all_filepaths.keys())

all_filepaths["top"], len(all_filepaths["top"])

In [ ]:
offs_idx = 1
for scene_id in range(1, 38):
    pcd_dir = f'/media/kluo/My Passport/v2x_dataset/MergedPointCloud/mini_{scene_id}/'

    all_files = []
    for f in os.listdir(pcd_dir):
        all_files.append(int(f.split("_")[-1][:-4]))
    
    print("***********scene:", scene_id)

    start_idx, stop_idx = min(all_files) + offs_idx, max(all_files) + offs_idx
    for idx in range(start_idx, stop_idx, 10):
        print(idx)

In [ ]:
# filename = "ca9a282c9e77460f8360f564131a8af5_nuscenes.bin"

# with open(f"path/to/{filename}", "rb") as f:
#     asset = client.upload_asset(f, filename=filename)

# point_cloud_url = asset.url

pcd_dir = f'/media/kluo/My Passport/v2x_dataset/MergedPointCloud/mini_5/'

all_files = []
for f in os.listdir(pcd_dir):
    all_files.append(int(f.split("_")[-1][:-4]))
        
all_point_cloud_urls = []
# for idx in range(len(all_filepaths["top"])):
# for idx in range(0, len(all_filepaths["top"]), 10):
start_idx, stop_idx = min(all_files) + offs_idx, max(all_files) + 1
print(start_idx, stop_idx)
for idx in range(start_idx, stop_idx, 10):
    filename = f"merged_{idx}.pcd"
    with open(osp.join(pcd_dir, filename), "rb") as f:
        asset = client.upload_asset(f, filename=filename)
    point_cloud_url = asset.url
    all_point_cloud_urls.append(point_cloud_url)

In [ ]:
all_point_cloud_urls

In [ ]:
# construct the frames

# -0.0389144089103	-0.023183467107	0.9989570081	0.00574816730405
# x, y,z,w
frames = []
str_idx = 0
for idx in range(start_idx, stop_idx, 10):
    print(idx)
    timestamp = int(all_filepaths["top"][str(idx)][1]*1e9)
    frame = {
        "pcd": {"url": all_point_cloud_urls[str_idx], "type": "pcd"},
        "timestamp": timestamp,
    }
    frames.append(frame)
    str_idx += 1

In [ ]:
frames

In [ ]:
len(frames)

In [ ]:
dataset = "katieluo6/TestingV2X"

name = "sample_point_cloud_sequence_v3"

attributes = {"frames": frames}

sample = client.add_sample(dataset, name, attributes)

# upload all files

In [ ]:
dataset = "julieberrioperez/V2X_dataset"

In [ ]:
offs_idx = 1

In [ ]:
scene_point_cloud_urls = {}

In [ ]:

for scene_id in range(1, 38):
    if scene_id == 18:
        print("skipping", scene_id)
        continue
    print("*************** scene id:", scene_id)
    pcd_path = f"/media/kluo/My Passport/v2x_dataset/PointCloud/mini_{scene_id}"
    odo_path = f"/media/kluo/My Passport/v2x_dataset/Odometry/mini_{scene_id}"

    # store all filepaths according to their header
    all_filepaths = {}
    for f in os.listdir(pcd_path):
        if f.split(".")[-1] != "pcd":
            continue
        sensor, ts, stamp = f.split("_")
        stamp = check_stamp(stamp[:-4])
        stamp = float(stamp)
        if sensor not in all_filepaths:
            all_filepaths[sensor] = {ts: (f, stamp)}
        else:
            all_filepaths[sensor][ts] = (f, stamp)

    pcd_dir = f'/media/kluo/My Passport/v2x_dataset/MergedPointCloud/mini_{scene_id}/'
    
    all_files = []
    for f in os.listdir(pcd_dir):
        all_files.append(int(f.split("_")[-1][:-4]))
    
    # start_idx, stop_idx = min(all_files), max(all_files)
    start_idx, stop_idx = min(all_files) + offs_idx, max(all_files) + 1
    
    all_point_cloud_urls = []
    for idx in range(start_idx, stop_idx, 10):
        filename = f"merged_{idx}.pcd"
        print(filename)
        with open(osp.join(pcd_dir, filename), "rb") as f:
            asset = client.upload_asset(f, filename=filename)
        point_cloud_url = asset.url
        print(idx, point_cloud_url)
        all_point_cloud_urls.append(point_cloud_url)
    scene_point_cloud_urls[scene_id] = all_point_cloud_urls

    # construct the frames
    # x, y,z,w
    frames = []
    str_idx = 0
    for idx in range(start_idx, stop_idx, 10):
        print(idx)
        timestamp = int(all_filepaths["top"][str(idx)][1]*1e9)
        frame = {
            "pcd": {"url": all_point_cloud_urls[str_idx], "type": "pcd"},
            "timestamp": timestamp,
        }
        frames.append(frame)
        str_idx += 1
    print("frames:", len(frames))
    name = f"point_cloud_sequence_{scene_id}"

    attributes = {"frames": frames}

    sample = client.add_sample(dataset, name, attributes)

In [ ]:
len(scene_point_cloud_urls)

In [ ]:
labeled_idxs = {}
for scene_id in range(1, 38):
    if scene_id == 18:
        print("skipping", scene_id)
        continue
    print("*************** scene id:", scene_id)

    pcd_dir = f'/media/kluo/My Passport/v2x_dataset/MergedPointCloud/mini_{scene_id}/'
    
    all_files = []
    for f in os.listdir(pcd_dir):
        all_files.append(int(f.split("_")[-1][:-4]))
    
    # start_idx, stop_idx = min(all_files), max(all_files)
    start_idx, stop_idx = min(all_files) + offs_idx, max(all_files) + 1
    
    for idx in range(start_idx, stop_idx, 10):
        if scene_id not in labeled_idxs:
            labeled_idxs[scene_id] = []
        labeled_idxs[scene_id].append(idx)


print(labeled_idxs)

In [ ]:
import json

In [ ]:
# save_dir = f"./merged_info/"
save_dir = f"/media/kluo/My Passport/v2x_dataset/merged_info_updated/"

scene_id = 5
timestep = 200

scene_info = []
with open(osp.join(save_dir, f"info_{scene_id}.jsonl"), "r") as file:
    for line in file:
        scene_info.append(json.loads(line))
len(scene_info), scene_info[timestep]

In [ ]:
offs = 1.712120e9
for scene_id in range(1, 37):

    scene_info = []
    with open(osp.join(save_dir, f"info_{scene_id}.jsonl"), "r") as file:
        for line in file:
            scene_info.append(json.loads(line))
            
    next_scene_info = []
    with open(osp.join(save_dir, f"info_{scene_id + 1}.jsonl"), "r") as file:
        for line in file:
            next_scene_info.append(json.loads(line))
    
    print(scene_info[-1]['dome']['timestamp'] - offs < next_scene_info[0]['dome']['timestamp'] - offs)

In [ ]:
# save with labeled boolean
for scene_id in range(1, 38):
    if scene_id == 18:
        print("skipping", scene_id)
        continue

    scene_info = []
    with open(osp.join(save_dir, f"info_{scene_id}.jsonl"), "r") as file:
        for line in file:
            timestep_scene_info = json.loads(line)
            idx = int(timestep_scene_info['dome']['pcd_filepath'].split("/")[-1].split("_")[1])
            is_labeled = idx in labeled_idxs[scene_id]
            timestep_scene_info["labeled"] = is_labeled
            print(idx, is_labeled)
            scene_info.append(timestep_scene_info)
    
    # Writing JSON lines to a file
    save_path = osp.join(save_dir, f"info_{scene_id}.jsonl")
    with open(save_path, 'w') as file:
        for entry in scene_info:
            json_line = json.dumps(entry)  # Convert dictionary to JSON string
            file.write(json_line + '\n')  # Write the JSON string to file and move to the next line

In [ ]:
save_dir